In [1]:
import sys
import torch
sys.path.append('../src')

from generator import *
from evaluation import *
from fair_model import FairModel
from baselines import LR, CvxFairModel, EOFairModel
from utils import gen_plot_data, plot_axes, combine_tuples

In [2]:
# Build Bank model
bank = Bank()
agent_train = Agent(n_samples=4000, protect_ratio=0.5, eps=0.5, base=[0.2, 1.0], seed=2021)
agent_test = Agent(n_samples=1000, protect_ratio=0.5, eps=0.5, base=[0.2, 1.0], seed=2020)

In [3]:
# Generate datasets
s_train, Xs_train, Ys_train = gen_multi_step_profiles(bank, agent_train, steps=5)
s_test, Xs_test, Ys_test = gen_multi_step_profiles(bank, agent_test, steps=5)
s_comb, X_comb, Y_comb = combine_tuples(s_train, Xs_train, Ys_train)

### Baseline: LR

In [4]:
lr = LR(l2_reg=1e-5)
lr.train(s_comb, X_comb, Y_comb)

_, Xs_te, Ys_te = gen_multi_step_profiles(lr, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, lr, OYs=OYs_te)

------------------------------ Step 1 - Logistic Regression ------------------------------
Acc: 91.2%
Short Fairness: 0.152
Long fairness: 0.058
------------------------------ Step 2 - Logistic Regression ------------------------------
Acc: 89.4%
Short Fairness: 0.160
Long fairness: 0.117
------------------------------ Step 3 - Logistic Regression ------------------------------
Acc: 91.7%
Short Fairness: 0.166
Long fairness: 0.173
------------------------------ Step 4 - Logistic Regression ------------------------------
Acc: 92.1%
Short Fairness: 0.164
Long fairness: 0.246
------------------------------ Step 5 - Logistic Regression ------------------------------
Acc: 91.7%
Short Fairness: 0.174
Long fairness: 0.340




### Baseline: FMDP

In [5]:
cfm = CvxFairModel(n_features=len(Xs_train[0][0])+2, l2_reg=1e-5, tao=1.565)
cfm.train(s_comb, X_comb, Y_comb)

_, Xs_te, Ys_te = gen_multi_step_profiles(cfm, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, cfm, OYs=OYs_te)

optimal
------------------------------ Step 1 - Fair Model with Demographic Parity ------------------------------
Acc: 73.5%
Short Fairness: 0.212
Long fairness: 0.180
------------------------------ Step 2 - Fair Model with Demographic Parity ------------------------------
Acc: 70.6%
Short Fairness: 0.216
Long fairness: 0.306
------------------------------ Step 3 - Fair Model with Demographic Parity ------------------------------
Acc: 70.4%
Short Fairness: 0.224
Long fairness: 0.376
------------------------------ Step 4 - Fair Model with Demographic Parity ------------------------------
Acc: 70.8%
Short Fairness: 0.220
Long fairness: 0.431
------------------------------ Step 5 - Fair Model with Demographic Parity ------------------------------
Acc: 72.5%
Short Fairness: 0.232
Long fairness: 0.481




## Baseline: FMEO

In [6]:
eqm = EOFairModel(n_features=len(Xs_train[0][0])+2, l2_reg=1e-5, tao=1.5)
eqm.train(s_comb, X_comb, Y_comb)

_, Xs_te, Ys_te = gen_multi_step_profiles(eqm, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, eqm, OYs=OYs_te)

optimal
------------------------------ Step 1 - Fair Model with Equal Oppertunity ------------------------------
Acc: 82.9%
Short Fairness: 0.010
Long fairness: 0.080
------------------------------ Step 2 - Fair Model with Equal Oppertunity ------------------------------
Acc: 79.0%
Short Fairness: 0.010
Long fairness: 0.122
------------------------------ Step 3 - Fair Model with Equal Oppertunity ------------------------------
Acc: 79.5%
Short Fairness: 0.010
Long fairness: 0.190
------------------------------ Step 4 - Fair Model with Equal Oppertunity ------------------------------
Acc: 80.0%
Short Fairness: 0.014
Long fairness: 0.276
------------------------------ Step 5 - Fair Model with Equal Oppertunity ------------------------------
Acc: 81.4%
Short Fairness: 0.020
Long fairness: 0.352




### Ours

In [7]:
fm = FairModel(n_features=len(Xs_train[0][0])+1, lr=5e-3, l2_reg=1e-5, sf_reg=0.119, lf_reg=0.154)
fm.train(s_train, Xs_train, Ys_train, Xs_train, Ys_train, epochs=1000, plot=False)

num_iters = 50

theta_true = fm.params
theta_list     = [np.copy(theta_true)]
theta_gaps     = []


# inital theta
theta = np.copy(theta_true)

for t in range(num_iters):
    # adjust distribution to current theta
    _, NXs_train, NYs_train = gen_multi_step_profiles(fm, agent_train, steps=5)
    # learn on induced distribution
    fm.train(s_train, Xs_train, Ys_train, NXs_train, NYs_train, epochs=10, plot=False)
    
    # keep track of statistic
    theta_new = fm.params
    theta_gaps.append(np.linalg.norm(theta_new - theta))
    theta_list.append(np.copy(theta_new))

    theta = np.copy(theta_new)
print("Retraining Done!")

Retraining Done!


In [8]:
_, Xs_te, Ys_te = gen_multi_step_profiles(fm, agent_test, steps=5)
OYs_te = generate_y_from_bank(s_test, Xs_te, bank)
compute_statistics(s_test, Xs_te, Ys_te, fm, OYs=OYs_te)

------------------------------ Step 1 - Long-term Fair Model ------------------------------
Acc: 80.1%
Short Fairness: 0.012
Long fairness: 0.040
------------------------------ Step 2 - Long-term Fair Model ------------------------------
Acc: 75.4%
Short Fairness: 0.008
Long fairness: 0.024
------------------------------ Step 3 - Long-term Fair Model ------------------------------
Acc: 72.9%
Short Fairness: 0.012
Long fairness: 0.020
------------------------------ Step 4 - Long-term Fair Model ------------------------------
Acc: 70.7%
Short Fairness: 0.008
Long fairness: 0.012
------------------------------ Step 5 - Long-term Fair Model ------------------------------
Acc: 69.2%
Short Fairness: 0.002
Long fairness: 0.002


